# This notebook perform simulations of phase transitions in cesium lead iodide (CsPbI3)

In [86]:
import numpy as np
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import view
from ase.lattice.surface import surface
from ase import Atoms
from ase.io import *
from ase.io import read, write
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.io.lammpsdata import write_lammps_data
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
import subprocess
import plumed
import nglview

# ortho-CsPbI3

In [311]:
unit_cell = read('ortho.cif')

rep1 = 2
rep2 = 2
rep3 = 2

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.75
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.75
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.50
    i = i + 1

supercell.set_initial_charges(charges=charge_array)

#view(supercell)
print(supercell) 

write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100 # ps

variable        pressure equal 1.00
variable        pressureDamp equal 500 #ps

variable        seed world 1428

read_data       data.CPI

mass            1 132.904999 # Cs
mass            2 126.90447  # I
mass            3 204.199997 # Pb

variable        freq equal 500

pair_style      lj/cut/coul/long 10.0 10.0
pair_modify     mix arithmetic
neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4
dielectric 1.0

variable AI     equal "0.042784417"
variable rI     equal "5.04975"
variable ACs    equal "0.0898565009" 
variable rCs    equal "3.521013"
variable ACsI   equal "0.0467571702"
variable rCsI   equal "4.43790"

variable APb    equal "0.0942780"
variable rPb    equal "4.043"
variable APbI   equal "4.7050"
variable rPbI   equal "2.800"
variable APbCs   equal "4.01050"
variable rPbCs   equal "4.100"

pair_coeff      1 1 ${ACs}    ${rCs} 
pair_coeff      2 2 ${AI}     ${rI}  
pair_coeff      3 3 ${APb}    ${rPb}     
pair_coeff      1 2 ${ACsI}   ${rCsI}    
pair_coeff      2 3 ${APbI}   ${rPbI}  
pair_coeff      1 3 ${APbCs}  ${rPbCs}   


thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization

# Minimization
min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-3 1.0e-5 1000 10000
unfix 1
write_data      data.min
reset_timestep  0

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.ortho.lammpstrj 

fix             1 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             2 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             3 all momentum 10000 linear 1 1 1

run             10000

unfix           1
unfix           2
unfix           3

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)


Atoms(symbols='Cs32I96Pb32', pbc=True, cell=[17.712, 17.152, 24.944], initial_charges=..., spacegroup_kinds=...)


In [312]:
subprocess.run("nohup lmp_intel_cpu_intelmpi < start.lmp &",shell=True)

CompletedProcess(args='nohup lmp_intel_cpu_intelmpi < start.lmp &', returncode=0)

LAMMPS (23 Jun 2022)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (17.712 17.152 24.944) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  160 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.004 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28324013
  grid = 12 12 15
  stencil order = 5
  estimated absolute RMS force accuracy = 0.012739991
  estimated relative force accuracy = 3.8366104e-05
  using single precision MKL FFT
  3d grid and FFT values/proc = 5780 2160
Generate

      9000   307.16057     -13691.196      145.57838     -13545.618      11773.698      16.337669      16.190651      23.277174      0              0              0            
      9500   269.58698     -13709.529      127.77043     -13581.759      11344.079      16.337669      16.190651      23.277174      0              0              0            
     10000   287.48019     -13684.598      136.25089     -13548.347      11402.248      16.337669      16.190651      23.277174      0              0              0            
Loop time of 4.74021 on 1 procs for 10000 steps with 160 atoms

Performance: 364.541 ns/day, 0.066 hours/ns, 2109.613 timesteps/s
97.5% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 2.0642     | 2.0642     | 2.0642     |   0.0 | 43.55
Bond    | 0.013649   | 0.013649   | 0.013649   |   0.0 |  0.29
Kspace 

# hexagonal CsPbI3

In [309]:
unit_cell = read('delta.cif')

rep1 = 2
rep2 = 4
rep3 = 1

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.75
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.75
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.50
    i = i + 1

supercell.set_initial_charges(charges=charge_array)

#view(supercell)
print(supercell) 

write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100 # ps

variable        pressure equal 1.00
variable        pressureDamp equal 500 #ps

variable        seed world 1428

read_data       data.CPI

mass            1 132.904999 # Cs
mass            2 126.90447  # I
mass            3 204.199997 # Pb

variable        freq equal 500


pair_style      lj/cut/coul/long 10.0 10.0
pair_modify     mix arithmetic
neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4
dielectric 1.0

variable AI     equal "0.042784417"
variable rI     equal "5.04975"
variable ACs    equal "0.0898565009" 
variable rCs    equal "3.521013"
variable ACsI   equal "0.0467571702"
variable rCsI   equal "4.43790"

variable APb    equal "0.0942780"
variable rPb    equal "4.043"
variable APbI   equal "4.7050"
variable rPbI   equal "2.800"

variable APbCs   equal "4.01050"
variable rPbCs   equal "4.100"

pair_coeff      1 1 ${ACs}    ${rCs} 
pair_coeff      2 2 ${AI}     ${rI}  
pair_coeff      3 3 ${APb}    ${rPb}     
pair_coeff      1 2 ${ACsI}   ${rCsI}    
pair_coeff      2 3 ${APbI}   ${rPbI}   
pair_coeff      1 3 ${APbCs}  ${rPbCs}   


thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization
min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-3 1.0e-5 1000 10000
unfix 1
write_data      data.min
reset_timestep  0

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.hexagonal.lammpstrj 

fix             1 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             2 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             3 all momentum 10000 linear 1 1 1

run             10000

unfix           1
unfix           2
unfix           3

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)


Atoms(symbols='Cs32I96Pb32', pbc=True, cell=[20.8684, 19.162, 17.761], initial_charges=..., spacegroup_kinds=...)


In [310]:
subprocess.run("nohup lmp_intel_cpu_intelmpi < start.lmp &",shell=True)

CompletedProcess(args='nohup lmp_intel_cpu_intelmpi < start.lmp &', returncode=0)

LAMMPS (23 Jun 2022)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (20.8684 19.162 17.761) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  160 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.004 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28350276
  grid = 15 12 12
  stencil order = 5
  estimated absolute RMS force accuracy = 0.012965236
  estimated relative force accuracy = 3.9044422e-05
  using single precision MKL FFT
  3d grid and FFT values/proc = 5780 2160
Generat

      9000   306.8843      -13193.345      145.44744     -13047.898      21187.609      19.378895      18.394819      16.489324      0              0              0            
      9500   249.30201     -13198.465      118.15639     -13080.309      21439.822      19.378895      18.394819      16.489324      0              0              0            
     10000   298.33086     -13189.941      141.39355     -13048.547      19413.485      19.378895      18.394819      16.489324      0              0              0            
Loop time of 4.73035 on 1 procs for 10000 steps with 160 atoms

Performance: 365.301 ns/day, 0.066 hours/ns, 2114.010 timesteps/s
97.6% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 2.1803     | 2.1803     | 2.1803     |   0.0 | 46.09
Bond    | 0.012829   | 0.012829   | 0.012829   |   0.0 |  0.27
Kspace 